In [ ]:
from tensorflow.keras.layers import Attention, Dropout, GroupNormalization, BatchNormalization, Flatten, Dense, Conv1D, MaxPooling1D,InputLayer, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import sklearn.linear_model
from PIL import Image
import tensorflow as tf
from google.colab import drive
from PIL import Image
drive.mount('/content/drive/')
DIR = 'drive/MyDrive/SHTEM24PAPAYA/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
def padding(small, img_back, hi = 1/5, vi = 1/5):
  small_dims = small.shape
  big_dims = img_back.shape
  horiz_pad = big_dims[1]-small_dims[1]
  vert_pad = big_dims[0]-small_dims[0]
  left_pad = int(np.round(horiz_pad*hi,0))
  up_pad = int(np.round(vert_pad*vi,0))
  padded = np.zeros_like(img_back)
  padded[up_pad:up_pad+small_dims[0], left_pad:left_pad+small_dims[1]] = small
  return padded

In [ ]:
x = np.load(DIR+'350x.npy')
y = np.load(DIR+'350y.npy')

In [ ]:
#WITH BACTHNORM
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.applications import DenseNet121

def build_densenet121_model(k, f, input_shape=(512, 512, 3)):

    base_model =  DenseNet121 (weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    encoder_output = base_model.output

    up1 = layers.UpSampling2D((2, 2))(encoder_output)
    conv1 = layers.Conv2D(f, (k, k), padding='same')(up1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.ELU()(conv1)
    conv1 = layers.Conv2D(f*2, (k, k), padding='same')(conv1)
    conv1 = layers.BatchNormalization()(conv1)
    conv1 = layers.ELU()(conv1)

    up2 = layers.UpSampling2D((2, 2))(conv1)
    conv2 = layers.Conv2D(f*2, (k, k), padding='same')(up2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.ELU()(conv2)
    conv2 = layers.Conv2D(f*4, (k, k), padding='same')(conv2)
    conv2 = layers.BatchNormalization()(conv2)
    conv2 = layers.ELU()(conv2)

    up3 = layers.UpSampling2D((2, 2))(conv2)
    conv3 = layers.Conv2D(f*4, (k, k), padding='same')(up3)
    conv3 = layers.BatchNormalization()(conv3)
    conv3 = layers.ELU()(conv3)

    up4 = layers.UpSampling2D((2, 2))(conv3)
    conv4 = layers.Conv2D(f*4, (k, k), padding='same')(up4)
    conv4 = layers.BatchNormalization()(conv4)
    conv4 = layers.ELU()(conv4)

    up5 = layers.UpSampling2D((2, 2))(conv4)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

for k in [3,4,5]:
    for f in [2,4,8]:
        dnet121Model = build_densenet121_model(k,f)

        dnet121Model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.BinaryIoU(
            target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
            )])

        dnet121Model.summary()

        trainnum = int(len(x)*0.9)
        e= dnet121Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)

        pd.DataFrame(e.history).to_csv(DIR+'Histories/dnet121Model_ELU_k'+str(k)+'_f'+str(f)+'_350_batchnorm.csv')

A saída de streaming foi truncada nas últimas 5000 linhas.
                                                                                                  
 pool3_conv (Conv2D)         (None, 64, 64, 256)          131072    ['pool3_relu[0][0]']          
                                                                                                  
 pool3_pool (AveragePooling  (None, 32, 32, 256)          0         ['pool3_conv[0][0]']          
 2D)                                                                                              
                                                                                                  
 conv4_block1_0_bn (BatchNo  (None, 32, 32, 256)          1024      ['pool3_pool[0][0]']          
 rmalization)                                                                                     
                                                                                                  
 conv4_block1_0_relu (Activ  (None, 32, 32, 256)  

In [ ]:
#WITHOUT BACTHNORM
 import tensorflow as tf
 from tensorflow.keras import layers, Model, Input
 from tensorflow.keras.applications import DenseNet121

 def build_densenet121_model(k, f, input_shape=(512, 512, 3)):

     base_model =  DenseNet121 (weights='imagenet', include_top=False, input_shape=input_shape)
     base_model.trainable = False
     encoder_output = base_model.output

     up1 = layers.UpSampling2D((2, 2))(encoder_output)
     conv1 = layers.Conv2D(f, (k, k), padding='same')(up1)
     conv1 = layers.ELU()(conv1)
     conv1 = layers.Conv2D(f*2, (k, k), padding='same')(conv1)
     conv1 = layers.ELU()(conv1)

     up2 = layers.UpSampling2D((2, 2))(conv1)
     conv2 = layers.Conv2D(f*2, (k, k), padding='same')(up2)
     conv2 = layers.ELU()(conv2)
     conv2 = layers.Conv2D(f*4, (k, k), padding='same')(conv2)
     conv2 = layers.ELU()(conv2)

     up3 = layers.UpSampling2D((2, 2))(conv2)
     conv3 = layers.Conv2D(f*4, (k, k), padding='same')(up3)
     conv3 = layers.ELU()(conv3)

     up4 = layers.UpSampling2D((2, 2))(conv3)
     conv4 = layers.Conv2D(f*4, (k, k), padding='same')(up4)
     conv4 = layers.ELU()(conv4)

     up5 = layers.UpSampling2D((2, 2))(conv4)
     outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(up5)

     model = Model(inputs=base_model.input, outputs=outputs)
     return model

 for k in [3,4,5]:
     for f in [2,4,8]:
         dnet121Model = build_densenet121_model(k,f)

         dnet121Model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=[tf.keras.metrics.BinaryIoU(
             target_class_ids=(0, 1), threshold=0.5, name=None, dtype=None
             )])

         dnet121Model.summary()

         trainnum = int(len(x)*0.9)
         e= dnet121Model.fit(x[:trainnum], y[:trainnum], validation_data=(x[trainnum:], y[trainnum:]), epochs=5, batch_size=32)
        pd.DataFrame(e.history).to_csv(DIR+'Histories/dnet121Model_ELU_k'+str(k)+'_f'+str(f)+'_350_nobatchnorm.csv')

A saída de streaming foi truncada nas últimas 5000 linhas.
 ation)                                                                                           
                                                                                                  
 conv3_block8_2_conv (Conv2  (None, 64, 64, 32)           36864     ['conv3_block8_1_relu[0][0]'] 
 D)                                                                                               
                                                                                                  
 conv3_block8_concat (Conca  (None, 64, 64, 384)          0         ['conv3_block7_concat[0][0]', 
 tenate)                                                             'conv3_block8_2_conv[0][0]'] 
                                                                                                  
 conv3_block9_0_bn (BatchNo  (None, 64, 64, 384)          1536      ['conv3_block8_concat[0][0]'] 
 rmalization)                                     